In [1]:
pwd

'h:\\My Drive\\projects\\scrapy-toy\\tgdd-selenium'

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException

from bs4 import BeautifulSoup

import json

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get('https://www.thegioididong.com/dtdd')
# driver.get('https://www.thegioididong.com/may-tinh-bang',)

In [5]:
# For category laptop-ldp

In [6]:
while True:
    try:
        button = driver.find_element(By.CLASS_NAME, "view-more")
        button.click()
    except (ElementNotInteractableException, ElementClickInterceptedException):
        break

In [7]:
# Extract product URLs from the category page
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
# Find all <a> elements with class "main-contain"
all_products = soup.find_all('a', class_='main-contain')
product_urls = ['https://thegioididong.com' + product['href'] for product in all_products]

In [9]:
product_urls, len(product_urls)

(['https://thegioididong.com/dtdd/iphone-15-pro-max?utm_flashsale=1',
  'https://thegioididong.com/dtdd/iphone-15-pro?utm_flashsale=1',
  'https://thegioididong.com/dtdd/iphone-15-plus?utm_flashsale=1',
  'https://thegioididong.com/dtdd/iphone-15?utm_flashsale=1',
  'https://thegioididong.com/dtdd/samsung-galaxy-s24-256gb-5g',
  'https://thegioididong.com/dtdd/samsung-galaxy-s24-plus-5g',
  'https://thegioididong.com/dtdd/samsung-galaxy-s24-ultra-5g',
  'https://thegioididong.com/dtdd/xiaomi-14',
  'https://thegioididong.com/dtdd/xiaomi-redmi-note-13-pro',
  'https://thegioididong.com/dtdd/oppo-reno11',
  'https://thegioididong.com/dtdd/xiaomi-redmi-a3',
  'https://thegioididong.com/dtdd/vivo-y03',
  'https://thegioididong.com/dtdd/realme-note-50-64gb',
  'https://thegioididong.com/dtdd/iphone-13?utm_flashsale=1',
  'https://thegioididong.com/dtdd/xiaomi-redmi-note-13-6gb-128gb',
  'https://thegioididong.com/dtdd/realme-c55',
  'https://thegioididong.com/dtdd/iphone-11?utm_flashsale=1'

In [10]:
from my_utils import save_list_to_txt_file

In [11]:
save_list_to_txt_file(product_urls, 'product_urls.txt')

Saved to product_urls.txt.


In [12]:
def extract_tech_specs(soup):
    # print(soup)
    
    big_types = soup.find_all(class_="parameter-ttl")
    # print(big_types)
    info_dict = {}
    for big_type in big_types:
        big_type_name = big_type.text.strip()
        info_dict[big_type_name] = {}
        sibling = big_type.find_next_sibling("ul")
        if sibling:
            for li in sibling.find_all("li"):
                smaller_type = li.find("div", class_="ctLeft").text.strip(": \n")
                detail_tag = li.find("div", class_="ctRight")
                circles = detail_tag.find_all("p", class_="circle")
                if circles:
                    detail = [circle.text.strip() for circle in circles]
                else:
                    detail = detail_tag.text.strip()
                info_dict[big_type_name][smaller_type] = detail
    # print(len(info_dict))
    return info_dict

In [17]:
for product_url in product_urls[:3]:
    
    # product_url += '#top-tskt'
    print(product_url)
    
    driver.get(product_url)

    # Find button for showing details of tech specification
    # button_chct = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn-detail.btn-short-spec.not-have-instruction")))
    button_chct = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn-detail.btn-short-spec")))
    button_chct.click()

    p_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//p[text()='Hãng:']")))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tech_specs = extract_tech_specs(soup)
    print(tech_specs)

    button = driver.find_element(By.ID, 'tab-article-gallery-0')
    button.click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    product_info = soup.find('div', class_='content-article').get_text()

    rating_tag = soup.find('div', class_='point')
    if rating_tag:
        rating = rating_tag.p.text
    else:
        rating = "Chưa có đánh giá"

    reviews_tag = soup.find('a', class_ = 'total-cmtrt')
    if reviews_tag:
        n_reviews = reviews_tag.text.replace('đánh giá', '').strip()
    else:
        n_reviews = 0

    print(rating, n_reviews)
    # print(product_info)

    # Find all tags with style="text-align: justify;"
    # tags_with_style = soup.find_all(lambda tag: tag.get('style') == "text-align: justify;")
    # # print(tags_with_style)

    # texts = [tag.get_text() for tag in tags_with_style]
    # product_info = '\n'.join(texts)


    # # Write product information to JSON
    # product_info = {
    #     'url': product_url,
    #     'tech_specs': tech_specs,
    #     'info': product_info
    # }

    
    # with open('products.json', 'a', encoding='utf-8') as outfile:
    #     json.dump(product_info, outfile, ensure_ascii=False)
    #     outfile.write('\n')

    # break


https://thegioididong.com/dtdd/iphone-15-pro-max?utm_flashsale=1
{'Màn hình': {'Công nghệ màn hình': 'OLED', 'Độ phân giải': 'Super Retina XDR (1290 x 2796 Pixels)', 'Màn hình rộng': '6.7" - Tần số quét 120 Hz', 'Độ sáng tối đa': '2000 nits', 'Mặt kính cảm ứng': 'Kính cường lực Ceramic Shield'}, 'Camera sau': {'Độ phân giải': 'Chính 48 MP & Phụ 12 MP, 12 MP', 'Quay phim': ['HD 720p@30fps', 'FullHD 1080p@60fps', 'FullHD 1080p@30fps', '4K 2160p@60fps', '4K 2160p@30fps'], 'Đèn Flash': 'Có', 'Tính năng': ['Ảnh Raw', 'Zoom quang học', 'Zoom kỹ thuật số', 'Xóa phông', 'Trôi nhanh thời gian (Time Lapse)', 'Toàn cảnh (Panorama)', 'Smart HDR 5', 'Siêu độ phân giải', 'Quay chậm (Slow Motion)', 'Live Photo', 'Góc siêu rộng (Ultrawide)', 'Góc rộng (Wide)', 'Dolby Vision HDR', 'Deep Fusion', 'Cinematic', 'Chống rung quang học (OIS)', 'Bộ lọc màu', 'Ban đêm (Night Mode)']}, 'Camera trước': {'Độ phân giải': '12 MP', 'Tính năng': ['Smart HDR 5', 'Xóa phông', 'Trôi nhanh thời gian (Time Lapse)', 'Quay

KeyboardInterrupt: 

In [13]:
def extract_general_inf(soup):
    pass

In [77]:
product_url = 'https://www.thegioididong.com/dtdd/iphone-11'
driver.get(product_url)
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [78]:
name = soup.find('section').find('h1').text
name

'Điện thoại iPhone 11 64GB'

In [15]:
colors_tag = soup.find('div', class_='box03 color group desk')
# print(colors_tag)

    
all_colors = [a_tag.get_text() for a_tag in colors_tag.find_all('a', class_='box03__item') ]
print(all_colors)

this_color = colors_tag.find('a', class_='box03__item item act').text
this_color

['Vàng', 'Xám', 'Đen', 'Tím']


'Vàng'

In [16]:
versions_tag = soup.find('div', class_='box03 group desk')
# print(colors_tag)

    
all_versions = [a_tag.get_text() for a_tag in versions_tag.find_all('a', class_='box03__item') ]
print(all_versions)

this_version = versions_tag.find('a', class_='box03__item item act').text
this_version

['256GB', '512GB']


'256GB'

In [23]:
# Price
price_tag = soup.find('div', class_='bs_price')
# print(price_tag)

promotion_price = price_tag.find('strong').text
print(promotion_price)

original_price = price_tag.find('em').text
print(original_price)

discount_percentage = price_tag.find('i').text.replace('(', '').replace(')', '').replace('-','')
print(discount_percentage)

promotion_end_date = soup.find('div', class_='bs_time').find_all('span')[1].text
print(promotion_end_date)

# original_price
# discount_percentage
# promotion_en_ddate

20.490.000₫
22.990.000₫
10%
23:00 | 30/04


In [60]:
policy_tag = soup.find('div', class_='policy')
# print(policy_tag)
a_tags = policy_tag.find_all('a')

# print(a_tags)

# print(len(a_tags))

# a_tag_texts = [a_tag.text.strip() for a_tag in a_tags if a_tag.text]
# print(a_tag_texts)

policies = [p_tag.text.replace('Xem chi tiết', '').replace('Xem địa chỉ bảo hành', '').replace('Xem hình', '').strip() for p_tag in policy_tag.find_all('p')]

for policy in policies:
    print(policy)


Hư gì đổi nấy 12 tháng  tại 3160 siêu thị toàn quốc (miễn phí tháng đầu)
Bảo hành chính hãng điện thoại 1 năm tại các trung tâm bảo hành hãng
Bộ sản phẩm gồm: Cáp Type C, Hộp, Sách hướng dẫn, Cây lấy sim


In [125]:
driver = webdriver.Chrome()

In [136]:
product_url = 'https://www.thegioididong.com/dtdd/oppo-reno11-f-5g'
driver.get(product_url)
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [139]:
def extract_general_info(soup):
    # Name
    name = soup.find('section').find('h1').text

    # Colors
    colors_tag = soup.find('div', class_='box03 color group desk')

    if colors_tag:
        all_colors = [a_tag.get_text() for a_tag in colors_tag.find_all('a', class_='box03__item')]
        this_color = colors_tag.find('a', class_='box03__item item act').text

    else:
        all_colors = this_color = None


    # Versions
    versions_tag = soup.find('div', class_='box03 group desk')

    if versions_tag:

        all_versions = [a_tag.get_text() for a_tag in versions_tag.find_all('a', class_='box03__item')]
        this_version = versions_tag.find('a', class_='box03__item item act').text

    else: 
        all_versions = this_version = None

    # Prices
    price_tag = soup.find('div', class_='bs_price')

    if price_tag:
        present_price = price_tag.find('strong').text
        original_price = price_tag.find('em').text
        discount_percentage = price_tag.find('i').text.replace('(', '').replace(')', '').replace('-', '')
        promotion_end_date = soup.find('div', class_='bs_time').find_all('span')[1].text
    
    else:
        
        price_tag = soup.find('div', class_='price-one')
        if price_tag:
            present_price = price_tag.find('p', class_='box-price-present').text.strip()
            
            original_price_tag = price_tag.find('p', class_='box-price-old')
            if original_price_tag:
                original_price = original_price_tag.text.strip()
            else: 
                original_price = None
                
            discount_percentage_tag = price_tag.find('p', class_='box-price-percent')
            if discount_percentage_tag:
                discount_percentage = discount_percentage_tag.text.strip().replace('-', '')
            else: 
                discount_percentage = None

            promotion_end_date_tag = soup.find('i', class_='pr-txt')
            if promotion_end_date_tag:
                promotion_end_date = promotion_end_date_tag.text
            else:
                promotion_end_date = None
        else:
            present_price = original_price = discount_percentage = promotion_end_date = None


    # Policies
    policy_tag = soup.find('div', class_='policy')
    policies = [p_tag.text.replace('Xem chi tiết', '').replace('Xem địa chỉ bảo hành', '').replace('Xem hình', '').strip() for p_tag in policy_tag.find_all('p')]

    # Promotion
    promotion_tags = soup.find_all('div', class_='divb-right')
    promotions = [p.text.replace("(Xem chi tiết tại đây)", '').replace("Xem chi tiết", '').strip() for p in promotion_tags]

    return {
        'name': name,
        'colors': {'all_colors': all_colors, 'this_color': this_color},
        'versions': {'all_versions': all_versions, 'this_version': this_version},
        'prices': {'present_price': present_price, 'original_price': original_price, 'discount_percentage': discount_percentage, 'promotion_end_date': promotion_end_date},
        'policies': policies,
        'promotions': promotions
    }


In [140]:
# Usage example:
extracted_info = extract_general_info(soup)
extracted_info

{'name': 'Điện thoại OPPO Reno11 F 5G Tím',
 'colors': {'all_colors': None, 'this_color': None},
 'versions': {'all_versions': ['Màu Đặc Quyền', 'Phiên Bản Chuẩn'],
  'this_version': 'Màu Đặc Quyền'},
 'prices': {'present_price': '8.990.000₫',
  'original_price': None,
  'discount_percentage': None,
  'promotion_end_date': 'Giá và khuyến mãi dự kiến áp dụng đến 23:00 | 01/05'},
 'policies': ['1 đổi 1 trong 30 ngày đối với sản phẩm lỗi  tại 3160 siêu thị toàn quốc',
  'Bảo hành chính hãng điện thoại 1 năm tại các trung tâm bảo hành hãng',
  'Bộ sản phẩm gồm: Hộp, Sách hướng dẫn, Cây lấy sim, Ốp lưng, Cáp Type C, Củ sạc nhanh rời đầu Type A'],
 'promotions': ['Phiếu mua hàng 300,000đ',
  'Thu cũ Đổi mới: Giảm đến 1 triệu (Tuỳ model máy cũ, Không kèm khuyến mãi giảm giá khác, cổng thanh toán, mua kèm)',
  'Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.000đ) khi thanh toán qua VNPAY-QR, áp dụng cho đơn hàng từ 3 Triệu']}

In [101]:
def generate_hierarchical_table(data):
    table_html = "<table>"
    for category, specs in data.items():
        table_html += f"<tr><th colspan='2'>{category}</th></tr>"
        for key, value in specs.items():
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    table_html += f"<tr><td>{sub_key}</td><td>{sub_value}</td></tr>"
            else:
                table_html += f"<tr><td>{key}</td><td>{value}</td></tr>"
    table_html += "</table>"
    return table_html

In [103]:
generate_hierarchical_table(extracted_info)

AttributeError: 'str' object has no attribute 'items'

In [96]:
# promotion_tag = soup.find('div', 'pr-item')
promotion_tags = soup.find_all('div', 'divb-right')
promotion_tags = [p.text.replace("(Xem chi tiết tại đây)", '').replace("Xem chi tiết", '').strip() for p in promotion_tags]
promotion_tags

['Ốp lưng chính hãng Apple giảm thêm 300K khi mua kèm iPhone (áp dụng tuỳ model)',
 'Thu cũ Đổi mới: Giảm đến 2 triệu (Tuỳ model máy cũ, Không kèm thanh toán qua cổng online, mua kèm)',
 'Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.000đ) khi thanh toán qua VNPAY-QR, áp dụng cho đơn hàng từ 3 Triệu']

In [28]:
from datetime import datetime

def get_current_date():
    """
    Get the current date in the format YYYY-MM-DD.

    Returns:
        str: The current date in the format YYYY-MM-DD.
    """
    return datetime.today().strftime('%Y%m%d')

# Example usage:
current_date = get_current_date()
print("Current date:", current_date)


Current date: 20240327
